In [61]:
import requests
from bs4 import BeautifulSoup
import time
import csv

filename = 'yelp_scrape_reviews.csv'
f = open(filename, 'w')
heads = 'Restaurant_Name,Food_Type,Price_Range,Average_Rating,Review_Count,Phone_Number,Address\n'
f.write(heads)

for start in range(0, 31, 10):

    # Sraping the URL
    url = f'https://www.yelp.com/search?find_desc=Restaurants&find_loc=London%2C%20United%20Kingdom&ns=1&={start}'

    headers = {'user-agent': 'Chrome/92.0.4515.131'}
    html_req = requests.get(url, headers=headers)

    time.sleep(10)

    soup = BeautifulSoup(html_req.content, 'lxml')

    names_lst = []
    content = soup.select('[class*=Name]')
    for item in content:
        name = item.text[3:].replace(', ', '-').strip()
        names_lst.append(name)

    price_lst = []
    cuisine_lst = []
    cuisine_price = soup.select('[class*=priceCategory]')
    for item in cuisine_price:
        cp_lst = item.text.split('$')
        price_lst.append(''.join(['$' for x in cp_lst if x == '']))
        cuisine_lst.append(cp_lst[-1].replace(', ', '-'))

    rating_lst = []
    ratings = soup.select('[aria-label*=rating]')
    for item in ratings:
        stars = item['aria-label'].split(' ')[0]
        rating_lst.append(f'{stars} stars')

    rev_count_lst = []
    rev_counts = soup.select('[class*=reviewCount]')
    for item in rev_counts:
        rev_count_lst.append(item.text)
        
    zipped_lst = [(a, b, c, d, e) for a, b, c, d, e in zip(names_lst,
                                                                       cuisine_lst,
                                                                       price_lst,
                                                                       rating_lst,
                                                                       rev_count_lst)]

    for i in zipped_lst:
        try:
            f.write(i[0] + ',' + i[1] + ',' + i[2] + ',' + i[3] + ',' + i[4] + ',' + '\n')
        except UnicodeEncodeError:
            pass

f.close()

In [54]:
names_lst

['The Mayfair Chippy',
 'Restaurant Gordon Ramsay',
 'The Fat Bear',
 'Dishoom',
 'Flat Iron',
 'Padella',
 'Ffiona’s Restaurant',
 'The Queens Arms',
 'Abeno',
 'The Golden Chippy']

In [55]:
rev_count_lst

['283', '206', '122', '1850', '381', '208', '266', '120', '102', '110']

In [56]:
cuisine_lst

['Fish & Chips££Mayfair',
 'FrenchBritish££££Chelsea',
 'American (Traditional)Soul FoodCajun/Creole££Blackfriars',
 'Indian££Covent Garden',
 'Steakhouses££Soho',
 'Italian££London Bridge',
 'British££Kensington',
 'BritishPubsGastropubs££Victoria',
 'Japanese££Bloomsbury',
 'Fish & Chips££Deptford']

In [57]:
zipped_lst

[('The Mayfair Chippy', 'Fish & Chips££Mayfair', '', '4.5 stars', '283'),
 ('Restaurant Gordon Ramsay',
  'FrenchBritish££££Chelsea',
  '',
  '4.5 stars',
  '206'),
 ('The Fat Bear',
  'American (Traditional)Soul FoodCajun/Creole££Blackfriars',
  '',
  '4.5 stars',
  '122'),
 ('Dishoom', 'Indian££Covent Garden', '', '4.5 stars', '1850'),
 ('Flat Iron', 'Steakhouses££Soho', '', '4.5 stars', '381'),
 ('Padella', 'Italian££London Bridge', '', '4.5 stars', '208'),
 ('Ffiona’s Restaurant', 'British££Kensington', '', '4.5 stars', '266'),
 ('The Queens Arms',
  'BritishPubsGastropubs££Victoria',
  '',
  '4.5 stars',
  '120'),
 ('Abeno', 'Japanese££Bloomsbury', '', '4.5 stars', '102'),
 ('The Golden Chippy', 'Fish & Chips££Deptford', '', '5 stars', '110')]

In [58]:
import pandas as pd
df = pd.DataFrame(zipped_lst)

In [62]:
df

,0,1,2,3,4
0,The Mayfair Chippy,Fish & Chips££Mayfair,,4.5 stars,283
1,Restaurant Gordon Ramsay,FrenchBritish££££Chelsea,,4.5 stars,206
2,The Fat Bear,American (Traditional)Soul FoodCajun/Creole££B...,,4.5 stars,122
3,Dishoom,Indian££Covent Garden,,4.5 stars,1850
4,Flat Iron,Steakhouses££Soho,,4.5 stars,381
5,Padella,Italian££London Bridge,,4.5 stars,208
6,Ffiona’s Restaurant,British££Kensington,,4.5 stars,266
7,The Queens Arms,BritishPubsGastropubs££Victoria,,4.5 stars,120
8,Abeno,Japanese££Bloomsbury,,4.5 stars,102
9,The Golden Chippy,Fish & Chips££Deptford,,5 stars,110


In [ ]:
df.to_csv("Output.csv", columns = header)